In [4]:
import spacy
import pandas as pd
from nltk import edit_distance
import tqdm

In [2]:
nlp = spacy.load("it_core_news_lg")

In [3]:
df = pd.read_excel("ita_morph_pos.xlsx")
df

,Unnamed: 0,fqTot,gramCat,lemma,word,Phones,morph,pos,Gender,Number,...,Person,Tense,VerbForm,Clitic,PronType,Foreign,Degree,NumType,Definite,Poss
0,0,50518,P,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,5219,P IN P@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1544,P IN B@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,272,P IN C@,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,234,S,a,a,a,NaN,ADP,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73805,119992,28,E,zurigo,zurigo,ZuriGo,NaN,PROPN,0,0,...,0,0,0,0,0,0,0,0,0,0
73806,119994,2,E,zurletti,zurletti,Zurletti,Gender=Masc|Number=Plur|PronType=Ind,PRON,Masc,Plur,...,0,0,0,0,Ind,0,0,0,0,0
73807,119995,5,E,zurli,zurli,Zurli,Clitic=Yes|Gender=Masc|Number=Plur|Person=3|Pr...,VERB,Masc,Plur,...,3,0,Inf,Yes,Prs,0,0,0,0,0
73808,119996,2,E IN E@,zurli',zurli',Zurli,Clitic=Yes|Gender=Masc|Number=Plur|Person=3|Pr...,VERB,Masc,Plur,...,3,0,Inf,Yes,Prs,0,0,0,0,0


In [31]:
memo = {}

def max_prefix(s1, s2):
    i = 0
    l1, l2 = len(s1), len(s2)
    try:
        while s1[i] == s2[i]:
            i += 1
        return i
    except:
        return i
    
def find_closest_lemma(goal_lemma):
    if goal_lemma in memo:
        return memo[goal_lemma]
    # takes too long
    dist = df['lemma'].apply(lambda l: -max_prefix(l, goal_lemma))
    mins = df[dist == dist.min()]
#     print(mins)
    memo[goal_lemma] = mins.loc[mins['lemma'].apply(len).idxmin(), 'lemma']
    return memo[goal_lemma]
    

In [40]:
suffixes = ['eggi', 'ific', 'izz', 'aggi', 'aggin', 'ai', 'aiol', 'anz', 'enz', 'eri', 'ezz', 'i\'', 'ier', 'ifici', 'in', 'ism', 'esim', 'ist', 'ita\'', 'eta\'', 'ta\'', 'ituden', 'etudin', 'izi', 'uzi', 'ment', 'toi', 'tor', 'sor', 'tric', 'tori', 'ur', 'zion', 'sion', 'ace', 'al', 'ar', 'ari', 'atori', 'bil', 'es', 'an', 'in', 'asc', 'esc', 'evol', 'ic', 'ier', 'ifer', 'ific', 'efic', 'il', 'iv', 'izi', 'os', 'str']

df['actual_lemma'] = ''
df['derivational'] = ''

for idx, row in tqdm.tqdm(df.iterrows()):
    w = row['word']
    try:
        for s in suffixes:
            if s in w:
                goal_lemma = w[:w.index(s)]
                if lemma := find_closest_lemma(goal_lemma):
    #                 print(goal_lemma, lemma)
    #                 input()
                    df.loc[idx, 'actual_lemma'] = lemma
                    df.loc[idx, 'derivational'] = s
                break
        for p in prefixes:
            ...
    except Exception as e:
        print(e, idx, row)

26891it [00:03, 7688.86it/s]

argument of type 'bool' is not iterable 25507 Unnamed: 0           41364
fqTot                   43
gramCat                  G
lemma                falso
word                 False
Phones               False
morph                  NaN
pos                    NaN
Gender                   0
Number                   0
Mood                     0
Person                   0
Tense                    0
VerbForm                 0
Clitic                   0
PronType                 0
Foreign                  0
Degree                   0
NumType                  0
Definite                 0
Poss                     0
actual_lemma              
derivational              
actual_lemma_pref         
derivational_pref         
Name: 25507, dtype: object


73810it [00:09, 8177.01it/s]


In [41]:
prefixes = ['in', 'inn', 'im', 'ir', 'il']
df['actual_lemma_pref'] = ''
df['derivational_pref'] = ''

for idx, row in tqdm.tqdm(df.iterrows()):
    w = row['word']
    try:
        for p in prefixes:
            if w.startswith(p):
                goal_lemma = w[len(p):]
                if lemma := find_closest_lemma(goal_lemma):
#                     print(goal_lemma, lemma)
#                     input()
                    df.loc[idx, 'actual_lemma_pref'] = lemma
                    df.loc[idx, 'derivational_pref'] = p
                break
            elif w.startswith('a'):
                ...
        for p in prefixes:
            ...
    except Exception as e:
        print(e, idx, row)

29721it [00:01, 28936.34it/s]

'bool' object has no attribute 'startswith' 25507 Unnamed: 0           41364
fqTot                   43
gramCat                  G
lemma                falso
word                 False
Phones               False
morph                  NaN
pos                    NaN
Gender                   0
Number                   0
Mood                     0
Person                   0
Tense                    0
VerbForm                 0
Clitic                   0
PronType                 0
Foreign                  0
Degree                   0
NumType                  0
Definite                 0
Poss                     0
actual_lemma              
derivational              
actual_lemma_pref         
derivational_pref         
Name: 25507, dtype: object


73810it [00:03, 23597.17it/s]


In [42]:
df.to_excel("ita_deriv.xlsx")